Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).

In [0]:
!ls


### **этот модуль делаем на компе, загружаем готовый файл** ".conllu"

In [0]:
#'этот модуль делаем на комппе, после чего загружаем готовый файл testset2.conllu в colab
!C:\Users\Administrator\NLP_Admin\udpipe-1.2.0-bin\bin-osx\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
D:\__NLP\Model_UDPipe\russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

In [1]:

with open('testset2.conllu', 'r', encoding='utf-8') as f: #, encoding='utf-8'
    text = f.read()
print(len(text))

12434505


In [2]:
from nltk.parse import DependencyGraph


In [3]:
trees = []

with open('testset2.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [4]:
g = DependencyGraph(trees[8], top_relation_label='root')

In [5]:
_FILTER_RELS = ['obj']
def get_subtree(nodes, node):
    res=[]
    if 'deps' in nodes[node]:
        for rel in nodes[node]['deps']:
            if rel in _FILTER_RELS:
                for dep in nodes[node]['deps'][rel]:
                    res.append(dep)
    return res          
                          

In [6]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [7]:
result = []
verbs_count={}
verbs=[]
N=0
for t in trees:
  try:
    g = DependencyGraph(t, top_relation_label='root')
    np_list = []
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
          verb=g.nodes[n]['word'].lower()
          verbs.append(verb)
          
          np = list(sorted(get_subtree(g.nodes, n)))
          for i in np:
            w2=g.nodes[n]['word']+' '+g.nodes[i]['word']
            np_list.append((g.nodes[n]['word'], g.nodes[i]['word']))
    
    if len(np_list)>0:
      for el in np_list:
        result.append(el)
  except:
    print('ERROR:', t)

#verbs=set(verbs)
#print(verbs_count)
#print(verbs)
result

ERROR: 1	15	15	NUM	_	_	7	nummod	_	_
2	ноября	ноябрь	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	1	flat	_	_
3	поиску	поиск	NOUN	_	Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing	7	iobj	_	_
4	Росздравнадзора	Росздравнадзор	PROPN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	3	nmod	_	_
5	компания	компания	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	7	nsubj:pass	_	_
6	была	быть	AUX	_	Aspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	7	aux:pass	_	_
7	оштрафована	штрафовать	VERB	_	Aspect=Perf|Gender=Fem|Number=Sing|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass	0	root	_	_
8	на	на	ADP	_	_	10	case	_	_
9	40 000	40 000	NUM	_	_	10	nummod	_	_
10	руб	рубль	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur	7	obl	_	SpaceAfter=No
11	.	.	PUNCT	_	_	10	punct	_	SpacesAfter=\s\n
12	159	159	NUM	_	_	10	nummod	_	_
13	(	(	PUNCT	_	_	14	punct	_	SpaceAfter=No
14	мошенничество	мошенничество	NOUN	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	10	parataxis	_	_
15	в	

ERROR: 1	По	по	ADP	_	_	2	case	_	_
2	информации	информация	NOUN	_	Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing	8	parataxis	_	_
3	LV	LV	PROPN	_	Foreign=Yes	2	flat:foreign	_	_
4	Finance	Finance	PROPN	_	Foreign=Yes	2	flat:foreign	_	SpaceAfter=No
5	,	,	PUNCT	_	_	2	punct	_	_
6	суд	суд	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	8	nsubj	_	_
7	также	также	ADV	_	Degree=Pos	8	advmod	_	_
8	обязал	обязывать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
9	IPOC	IPOC	PROPN	_	Foreign=Yes	8	flat:foreign	_	_
10	оплатить	оплатить	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	8	xcomp	_	_
11	стоимость	стоимость	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	10	obj	_	_
12	разбирательства	разбирательство	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	11	nmod	_	_
13	–	–	PUNCT	_	_	17	punct	_	_
14	около	около	ADP	_	_	17	case	_	_
15	100 000	100 000	NUM	_	_	17	nummod	_	_
16	швейцарских	швейцарский	ADJ	_	Case=Gen|Degree=Pos|Number=Plur	17	amod	_	_


[('заявил', 'Ъ'),
 ('подал', 'иск'),
 ('признать', 'договор'),
 ('запускать', 'механизм'),
 ('рассмотрит', 'вопрос'),
 ('отстранил', 'его'),
 ('терпящих', 'бедствие'),
 ('определят', 'срок'),
 ('обвинили', 'которого'),
 ('оберегали', 'покой'),
 ('обвинили', 'руководство'),
 ('вызвали', 'недовольство'),
 ('привело', 'обитателей'),
 ('гласит', 'постановление'),
 ('приносила', 'пользу'),
 ('задержали', 'его'),
 ('уберегло', 'его'),
 ('получили', 'сроки'),
 ('собиравший', 'доказательства'),
 ('обвинивший', 'олигарха'),
 ('предали', 'суду'),
 ('предали', 'роту'),
 ('оплачивать', 'обязанность'),
 ('обвинил', 'Юлию'),
 ('Выиграть', 'суд'),
 ('направило', 'ведомства'),
 ('девоенизировать', 'суды'),
 ('имеет', '|'),
 ('просил', 'суд'),
 ('освободить', 'его'),
 ('сделал', 'заявление'),
 ('рассматривал', 'ходатайство'),
 ('признал', '|'),
 ('затрудняет', 'доступ'),
 ('отказал', 'жалобы'),
 ('отозвала', 'иск'),
 ('сделала', 'шаг'),
 ('отозвала', 'иск'),
 ('предоставлять', 'данные'),
 ('затевать', 

In [8]:
def select_pair(pair, List_verbs):
  #print(len(pair),len(List_verbs))
  res=[]
  for p in pair:
    for x in p:
      word_pair=set(x.split())
      #print(word_pair)
      #print(word_pair.intersection(List_verbs))

      if len(word_pair.intersection(List_verbs))>0:
        res.append(p)
     
  return res


In [9]:
list_pair=select_pair(result, verbs)
list_pair[10:20]

[('обвинили', 'руководство'),
 ('вызвали', 'недовольство'),
 ('привело', 'обитателей'),
 ('гласит', 'постановление'),
 ('приносила', 'пользу'),
 ('задержали', 'его'),
 ('уберегло', 'его'),
 ('получили', 'сроки'),
 ('собиравший', 'доказательства'),
 ('обвинивший', 'олигарха')]

In [10]:
from collections import Counter
verbs_count = Counter(verbs)
#print(verbs_count)
verbs_more_50 = [ w for w in verbs_count.elements() if verbs_count[w] >= 50]

list_pair=select_pair(result, verbs_more_50)

verbs_more_50 =set(verbs_more_50)
#verbs_50 =Counter(verbs_more_50)
list_pair
#verbs_more_50

#print(list_pair)


[('заявил', 'Ъ'),
 ('подал', 'иск'),
 ('признать', 'договор'),
 ('обвинили', 'которого'),
 ('обвинили', 'руководство'),
 ('обвинил', 'Юлию'),
 ('признал', '|'),
 ('отказал', 'жалобы'),
 ('обвинив', 'его'),
 ('обвинила', 'господина'),
 ('обвинила', 'Березовского'),
 ('обвинила', 'Березовского'),
 ('вынес', 'постановление'),
 ('принял', 'решение'),
 ('признать', 'ответственность'),
 ('выплатить', 'компенсации'),
 ('воспрепятствует', 'является'),
 ('вынес', 'вердикт'),
 ('обвинив', 'его'),
 ('подала', 'иск'),
 ('подтвердил', 'правомочность'),
 ('удовлетворил', 'ходатайство'),
 ('подтвердил', 'доходы'),
 ('обвинили', 'сына'),
 ('подтвердил', 'решение'),
 ('удовлетворил', 'требования'),
 ('заявил', 'Ъ'),
 ('взыскать', 'деньги'),
 ('признал', 'постановление'),
 ('удовлетворил', 'иск'),
 ('отклонил', 'иск'),
 ('приговорил', 'Березовского'),
 ('вынес', 'приговор'),
 ('подтвердил', 'приказ'),
 ('обвинил', 'Киселева'),
 ('подал', 'иск'),
 ('заявил', 'ходатайство'),
 ('удовлетворил', 'иск'),
 ('п

In [12]:
from pymorphy2 import MorphAnalyzer

m = MorphAnalyzer()

In [13]:
#преобразуем пару глагол + сущ в первонач форму
verb_noun_norm=[]
#print(List_pair)
for p in list_pair:
  if len(p[0])>1 and len(p[1])>1: #исключем случайный мусор, который попал
    el1=m.parse(p[0])[0].normal_form
    el2=m.parse(p[1])[0].normal_form
    norm_pair=(el1,el2)
    verb_noun_norm.append(norm_pair)

len(verb_noun_norm)

1041

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')
#stop

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**2. Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).**

In [15]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [16]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    return lemmas

In [17]:
import nltk
from nltk import collocations 

In [19]:
with open('testset2.txt', 'r', encoding='utf-8') as f: #, encoding='utf-8'
    testset2 = f.read()
len(testset2)

1037635

In [20]:
finder_test = collocations.BigramCollocationFinder.from_documents([normalize(testset2)])

In [21]:
finder_test.apply_ngram_filter(lambda w1, w2: (w1, w2) not in verb_noun_norm)

**-log-likeLlihood**

In [22]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().likelihood_ratio)

[(('подать', 'иск'), 387.1438873886019),
 (('удовлетворить', 'иск'), 377.6583419074894),
 (('принять', 'решение'), 295.9808220095109),
 (('вынести', 'решение'), 249.45326927567072),
 (('удовлетворить', 'ходатайство'), 228.56414080178342),
 (('вынести', 'приговор'), 139.74755462972476),
 (('подать', 'апелляция'), 116.14125005005982),
 (('отклонить', 'иск'), 109.6079211501184),
 (('признать', 'правота'), 96.28722585810247),
 (('отклонить', 'жалоба'), 83.38273625071997),
 (('признать', 'что'), 81.2300930771208),
 (('подтвердить', 'законность'), 80.06330379242267),
 (('выплатить', 'компенсация'), 63.67058001617606),
 (('удовлетворить', 'жалоба'), 63.440096560366044),
 (('обвинить', 'он'), 60.93852079127934),
 (('вынести', 'постановление'), 60.428990233057405),
 (('вынести', 'вердикт'), 47.650205803336306),
 (('подать', 'жалоба'), 46.014624443582875),
 (('обвинить', 'она'), 44.83230718528459),
 (('сообщить', 'адвокат'), 44.69266641908054),
 (('признать', 'себя'), 41.644351499497425),
 (('пр

In [23]:
likelihood_100 = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 100)
#len(likelihood_100)

**-Dice**

In [24]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().dice)

[(('удовлетворить', 'ходатайство'), 0.17197452229299362),
 (('подать', 'иск'), 0.14403778040141677),
 (('удовлетворить', 'иск'), 0.14343707713125844),
 (('подтвердить', 'законность'), 0.12598425196850394),
 (('выплатить', 'компенсация'), 0.10687022900763359),
 (('вынести', 'приговор'), 0.10610079575596817),
 (('принять', 'решение'), 0.0903954802259887),
 (('подать', 'апелляция'), 0.08253968253968254),
 (('вынести', 'решение'), 0.08125577100646353),
 (('отклонить', 'жалоба'), 0.08088235294117647),
 (('отклонить', 'апелляция'), 0.07476635514018691),
 (('вынести', 'постановление'), 0.06521739130434782),
 (('выплатить', 'штраф'), 0.06201550387596899),
 (('выплатить', 'иркутянин'), 0.06060606060606061),
 (('удовлетворить', 'жалоба'), 0.05913978494623656),
 (('признать', 'правота'), 0.05649717514124294),
 (('вынести', 'вердикт'), 0.056338028169014086),
 (('отклонить', 'иск'), 0.053208137715179966),
 (('приговорить', 'саддам'), 0.047619047619047616),
 (('удовлетворить', 'требование'), 0.04666

In [25]:
dice_100 = finder_test.nbest(collocations.BigramAssocMeasures().dice, 100)
dice_100

[('удовлетворить', 'ходатайство'),
 ('подать', 'иск'),
 ('удовлетворить', 'иск'),
 ('подтвердить', 'законность'),
 ('выплатить', 'компенсация'),
 ('вынести', 'приговор'),
 ('принять', 'решение'),
 ('подать', 'апелляция'),
 ('вынести', 'решение'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'апелляция'),
 ('вынести', 'постановление'),
 ('выплатить', 'штраф'),
 ('выплатить', 'иркутянин'),
 ('удовлетворить', 'жалоба'),
 ('признать', 'правота'),
 ('вынести', 'вердикт'),
 ('отклонить', 'иск'),
 ('приговорить', 'саддам'),
 ('удовлетворить', 'требование'),
 ('принять', 'участие'),
 ('обязать', 'кашулинский'),
 ('подать', 'жалоба'),
 ('подтвердить', 'правомочность'),
 ('считать', 'себя'),
 ('подтвердить', 'правомерность'),
 ('отказать', 'мосметрострой'),
 ('приговорить', 'капитан'),
 ('сообщить', 'адвокат'),
 ('признать', 'себя'),
 ('отказать', 'сигма'),
 ('подтвердить', 'запрет'),
 ('обвинить', 'она'),
 ('подать', 'ходатайство'),
 ('признать', 'что'),
 ('обвинить', 'он'),
 ('обвинить', 'президен

**-PMI**

In [26]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().pmi)

[(('выплатить', 'иркутянин'), 11.097589811375766),
 (('приговорить', 'взяточница'), 10.131805526713677),
 (('отказать', 'мосметрострой'), 9.840201968683115),
 (('сообщить', 'уметалиев'), 9.336038578931287),
 (('подтвердить', 'правомочность'), 9.267867076289708),
 (('обязать', 'фссна'), 9.190699215767246),
 (('подтвердить', 'правомерность'), 8.58979517117707),
 (('подтвердить', 'законность'), 8.419870169734756),
 (('сообщить', 'рса'), 8.336038578931285),
 (('обвинить', 'богот'), 8.221072864810766),
 (('обвинить', 'канцлер'), 8.221072864810766),
 (('обвинить', 'миллионер'), 8.221072864810766),
 (('обвинить', 'молодогвардеец'), 8.221072864810765),
 (('обязать', 'кашулинский'), 8.190699215767246),
 (('приговорить', 'водитель'), 8.131805526713679),
 (('признать', 'хуцишвили'), 8.120309887875848),
 (('выплатить', 'дивиденд'), 8.097589811375766),
 (('признать', 'правота'), 7.857275482042054),
 (('отказать', 'сигма'), 7.840201968683113),
 (('выплатить', 'компенсация'), 7.838855542975596),
 (('

In [27]:
pmi_100 = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 100)
pmi_100

[('выплатить', 'иркутянин'),
 ('приговорить', 'взяточница'),
 ('отказать', 'мосметрострой'),
 ('сообщить', 'уметалиев'),
 ('подтвердить', 'правомочность'),
 ('обязать', 'фссна'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'законность'),
 ('сообщить', 'рса'),
 ('обвинить', 'богот'),
 ('обвинить', 'канцлер'),
 ('обвинить', 'миллионер'),
 ('обвинить', 'молодогвардеец'),
 ('обязать', 'кашулинский'),
 ('приговорить', 'водитель'),
 ('признать', 'хуцишвили'),
 ('выплатить', 'дивиденд'),
 ('признать', 'правота'),
 ('отказать', 'сигма'),
 ('выплатить', 'компенсация'),
 ('отклонить', 'апелляция'),
 ('сообщить', 'морщаков'),
 ('признать', 'бабаков'),
 ('признать', 'соучастие'),
 ('обвинить', 'калягин'),
 ('подать', 'апелляция'),
 ('удовлетворить', 'ходатайство'),
 ('подтвердить', 'план'),
 ('обвинить', 'киселев'),
 ('обвинить', 'крайком'),
 ('обвинить', 'моряк'),
 ('приговорить', 'основатель'),
 ('отклонить', 'большинство'),
 ('удовлетворить', 'прошение'),
 ('выплатить', 'штраф'),
 ('под

**3. Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.**

In [28]:
GS_coll=set(likelihood_100).intersection(dice_100)
GS_coll=set(GS_coll).intersection(pmi_100)
GS_coll

{('взыскать', 'деньга'),
 ('вынести', 'вердикт'),
 ('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('выплатить', 'деньга'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'иркутянин'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('обвинить', 'моряк'),
 ('обязать', 'кашулинский'),
 ('обязать', 'структура'),
 ('обязать', 'фссна'),
 ('отказать', 'мосметрострой'),
 ('отказать', 'сигма'),
 ('отклонить', 'апелляция'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'иск'),
 ('отклонить', 'ходатайство'),
 ('подать', 'апелляция'),
 ('подать', 'иск'),
 ('подать', 'кассация'),
 ('подтвердить', 'законность'),
 ('подтвердить', 'запрет'),
 ('подтвердить', 'информация'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'правомочность'),
 ('приговорить', 'взяточница'),
 ('приговорить', 'водитель'),
 ('приговорить', 'капитан'),
 ('приговорить', 'основатель'),
 ('приговорить', 'саддам'),
 ('признать', 'забастовка'),
 ('признать', 'прав

In [30]:
import re

with open('verb_coll.txt', 'r', encoding='utf-8') as f:
    verb_txt = f.read()

# из корпуса вытаскиваем пары verb-noun
verb_coll=[]
pattern='VN.*\n'   
match = re.findall(pattern, verb_txt) 
for s in match:
    s=s.replace('VN','')
    s = re.sub(r'\s+', ' ', s, flags=re.UNICODE)
    s=s.replace('\n','')
    
    words = s.split()
    if len(words)==2:
        coll_lemm = (m.parse(words[0])[0].normal_form, m.parse(words[1])[0].normal_form)
        verb_coll.append(coll_lemm)
#print(verb_coll)

In [57]:
GS = set(GS_coll).intersection(set(verb_coll))
GS

{('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('подать', 'иск'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('удовлетворить', 'иск'),
 ('удовлетворить', 'требование')}

**4. Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.**

In [0]:
add_coll={
 ('вынести', 'вердикт'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('отклонить', 'апелляция'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'иск'),
 ('отклонить', 'ходатайство'),
 ('подать', 'апелляция'),
 ('подать', 'иск'),
 ('подать', 'кассация'),
 ('подтвердить', 'законность'),
 ('подтвердить', 'запрет'),
 ('подтвердить', 'информация'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'правомочность'),
 ('признать', 'правота'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('удовлетворить', 'жалоба'),
 ('удовлетворить', 'ходатайство')}
 

In [59]:
GS=GS.union(add_coll)
len(GS) #размер множества для расчета корреляций
#GS

27

**5. Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. Можно использовать scipy.stats.spearmanr. Опишите ошибки каждой метрики.**

In [0]:
from scipy.stats import spearmanr

In [0]:
like_27 = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 27)
dice_27 = finder_test.nbest(collocations.BigramAssocMeasures().dice, 27)
pmi_27 = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 27)


In [0]:
finder_test.apply_ngram_filter(lambda w1, w2: (w1, w2) not in GS)

In [0]:
def transform_to_row(list):
    row = []
    for el in list:
        row.append(' '.join(el))
    return row

In [0]:
row=transform_to_row(like_27)
row

**likelihood spearmanr**

In [0]:
like_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 27)
rho, p = spearmanr(transform_to_row(like_27_GS), transform_to_row(like_27))
print(f'rho_like: {rho}\np_like: {p}')
#like_27_GS

rho_like: 0.34859584859584863
p_like: 0.07474551078913241


In [77]:
dice_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().dice, 27)
rho, p = spearmanr(transform_to_row(dice_27_GS), transform_to_row(dice_27))
print(f'rho_dice: {rho}\np_dice: {p}')


rho_dice: 0.5079365079365078
p_dice: 0.006833748394532331


In [78]:
pmi_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 27)
rho, p = spearmanr(transform_to_row(pmi_27_GS), transform_to_row(pmi_27))
print(f'rho_pmi: {rho}\np_pmi: {p}')


rho_pmi: 0.2442002442002442
p_pmi: 0.21961664482521304
